<a href="https://colab.research.google.com/github/stephenfrein/csc8490/blob/main/ExternalApplicationtoOracle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage 1: Basic Connection and Operations

In [ ]:
# PIP is Python Improved Packaging
# command below installs a Python code package that allows to connect to Oracle easily
# the ! means "run this from the command line rather than as a python command"
!pip install oracledb

In [ ]:
# import allows us to load an available package into memory so we can use it
import oracledb

# set up our connection information - enter YOUR regular Oracle credentials here
db_user = ""
db_password = ""
connect_string = "vu2025.cypibltd7eim.us-east-2.rds.amazonaws.com/ORCL"

# make a connection - this creates a session with the database
conn = oracledb.connect(user=db_user, password=db_password, dsn=connect_string)

# prove that we've made the connection
print("Database version:", conn.version)

In [ ]:
# select some data from the database
# a cursor in this case is similar to a PL/SQL cursor - a memory area used for processing SQL
cursor = conn.cursor()
# for each iteration of the for loop, result gets populated with the next row from the query results
for result in cursor.execute("select * from HR.Countries"):
    print(result)


In [ ]:
# calling a stored procedure
# run the stored procedure setup in the slides first
# set up a variable to accept the procedure output
result = cursor.var(int)
# calling the procedure
cursor.callproc("DOUBLER", [6, result])
# show result
print(result)
# show just the value frrom result
print(result.getvalue())

In [ ]:
# insert some records
# assumes you've created the ExtAppsTest table using the slides
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (1, 'Blue', 14)")
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (2, 'Red', 16)")
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (3, 'Blue', 19)")
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (4, 'Yellow', 20)")
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (5, 'Red', 22)")



In [ ]:
# see the data you just entered
for result in cursor.execute("select * from ExtAppsTest"):
    print(result)

In [ ]:
# update one of the records
cursor.execute("update ExtAppsTest set TextCol = 'Purple' where IdVal = 5")

In [ ]:
for result in cursor.execute("select * from ExtAppsTest"):
    print(result)

In [ ]:
# make changes public and permanent
conn.commit()

In [ ]:
# clears out records
cursor.execute("delete from ExtAppsTest")

In [ ]:
# close both the cursor and the connection to free up resources
cursor.close()
conn.close()

# Stage 2: Externalize Secrets and Environment-Specific Configuration

In [ ]:
# condensed example of code we looked at in Stage 1
import oracledb
# set up connection information using YOUR credentials
db_user = ""
db_password = ""
connect_string = "vu2025.cypibltd7eim.us-east-2.rds.amazonaws.com/ORCL"
# make a connection - this creates a session with the database
conn = oracledb.connect(user=db_user, password=db_password, dsn=connect_string)
# select some data from the database
cursor = conn.cursor()
for result in cursor.execute("select * from HR.Countries where region_id = 1"):
    print(result)
cursor.close()
conn.close()

This is bad because we have username and password details directly in our code. Anybody with access to the source code can now login to the database with the exposed credentials. We also might want to use a test database in some environments and save the production database for production use only. So, we need to separate our secrets and environment-specific config from the code.

In [ ]:
# get rid of .env file if already exists
!rm .env

In [ ]:
# write username, password, and database server to .env file
!echo "username=" >> .env
!echo "password=" >> .env
!echo "connect_string=vu2025.cypibltd7eim.us-east-2.rds.amazonaws.com/ORCL" >> .env

In [ ]:
# check contents of file
!cat .env

In [ ]:
# install new package that allows us to get values from .env files like this
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os
# load those values from the .env file into environment variables
load_dotenv()
# condensed example from Stage 1 revised to externalize secrets and configuration details
import oracledb
# make a connection using the environment variables (our secrets are no longer in the code)
conn = oracledb.connect(user=os.getenv("username"), password=os.getenv("password"), dsn=os.getenv("connect_string"))
# select some data from the database
cursor = conn.cursor()
for result in cursor.execute("select * from HR.Countries where region_id = 1"):
    print(result)
cursor.close()
conn.close()

# Stage 3: Parameterize Commands for Security and Performance (Prepared Statements)

Here is a simple web app that queries the database by the employee id passed in the URL. We can only use it to see employee last names. It certainly won't show us sensitive information like those juicy vault combinatiions!

In [ ]:
    from google.colab import output
    output.serve_kernel_port_as_window(8084)

In [ ]:
    from flask import Flask
    from flask import jsonify
    from flask import request
    from dotenv import load_dotenv
    import os
    import oracledb
    app = Flask(__name__)

    # routes match url paths users enter and take action on them
    @app.route("/")
    def hello():
        return "Hello, World!"

     # routes match url paths users enter and take action on them
    @app.route("/query")
    def query():
        # get the id parameter from the URL
        id = request.args.get('id')
        # load those values from the .env file into environment variables
        load_dotenv()
        # make a connection - this creates a session with the database
        conn = oracledb.connect(user=os.getenv("username"), password=os.getenv("password"), dsn=os.getenv("connect_string"))
        # create cursor for SQL processing
        cursor = conn.cursor()
        # execute a query using the parameter passed in through the URL
        print("SQL query: " + "select last_name from hr.employees where employee_id = " + str(id))
        cursor.execute("select last_name from hr.employees where employee_id = " + str(id))
        result = cursor.fetchone()
        # close both the cursor and the connection to free up resources
        cursor.close()
        conn.close()
        # show result on web page
        return str(result[0])

    if __name__ == "__main__":
        app.run(host='0.0.0.0', port=8084)

Now, try with URLs like /query?id=100 (remember that those employees have ids starting with 100)

Let's try some others...

/query?id=abc

/query?id=1 union select combo from vaultcombinations where vaultnumber = 1000

Whoa! What happened? This app has a SQL injection vulnerability! Let's fix that by using a prepared statement, which is a parameterized version of our query.

In [ ]:
    from flask import Flask
    from flask import jsonify
    from flask import request
    from dotenv import load_dotenv
    import os
    import oracledb
    app = Flask(__name__)

    @app.route("/")
    def hello():
        return "Hello, World!"

    @app.route("/query")
    def query():
        # get the id parameter from the URL
        id = request.args.get('id')
        # load those values from the .env file into environment variables
        load_dotenv()
        # make a connection - this creates a session with the database
        conn = oracledb.connect(user=os.getenv("username"), password=os.getenv("password"), dsn=os.getenv("connect_string"))
        # create cursor for SQL processing
        cursor = conn.cursor()
        # execute a query using the parameter passed in through the URL
        # this time, we parameterize our query using a "prepared statement" template and a bind variable
        # add a placeholder to the sql statement with a colon and an identifier, and then pass in the values to be used in those places
        # last argument has to be in array form - this is why we have parentheses and a comma after, like this: (id,)
        cursor.execute("select last_name from hr.employees where employee_id = :empid", empid=id)
        result = cursor.fetchone()
        # close both the cursor and the connection to free up resources
        cursor.close()
        conn.close()
        # show result on web page
        return str(result[0])

    if __name__ == "__main__":
        app.run(host='0.0.0.0', port=8084)

Let's try those tests again...

/query?id=100

/query?id=abc

/query?id=1 union select combo from vaultcombinations where vaultnumber = 1000

And now let's see the paramterized versions of all those basic operations we explored in Stage 1.

In [ ]:
from dotenv import load_dotenv
import os
# load those values from the .env file into environment variables
load_dotenv()
# condensed example from Stage 1 revised to externalize secrets and configuration details
import oracledb
# make a connection using the environment variables (our secrets are no longer in the code)
conn = oracledb.connect(user=os.getenv("username"), password=os.getenv("password"), dsn=os.getenv("connect_string"))
# select some data from the database
cursor = conn.cursor()
target_region = 3
for result in cursor.execute("select * from HR.Countries where region_id = :rgn",rgn=target_region):
    print(result)

In [ ]:
# insert some records
# assumes you've created the ExtAppsTest table using the slides
# using a different style for this one where each new row is represented as an array of values
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (:1, :2, :3)", (1, 'Blue', 14))
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (:1, :2, :3)", (2, 'Red', 16))
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (:1, :2, :3)", (3, 'Blue', 19))
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (:1, :2, :3)", (4, 'Yellow', 20))
cursor.execute("insert into ExtAppsTest (IdVal, TextCol, NumCol) values (:1, :2, :3)", (5, 'Red', 22))



In [ ]:
for result in cursor.execute("select * from ExtAppsTest"):
    print(result)

In [ ]:
# update one of the records
cursor.execute("update ExtAppsTest set TextCol = :color where IdVal = :id", color="Purple", id=5)

In [ ]:
for result in cursor.execute("select * from ExtAppsTest"):
    print(result)

In [ ]:
# can we pass in a column dynamically? (nope - will fail)
cursor.execute("update ExtAppsTest set :1 = :2 where IdVal = :3", ('TextCol','Magenta', 5))

In [ ]:
# close both the cursor and the connection to free up resources
cursor.close()
conn.close()

Quick performance test - let's see how the use of bind variables compares to using literal values.

We'll try literal values first.

***Need somebody to tell a bad joke while it runs.***

In [ ]:
from dotenv import load_dotenv
import os
import time

# load those values from the .env file into environment variables
load_dotenv()
# condensed example from Stage 1 revised to externalize secrets and configuration details
import oracledb
# make a connection using the environment variables (our secrets are no longer in the code)
conn = oracledb.connect(user=os.getenv("username"), password=os.getenv("password"), dsn=os.getenv("connect_string"))
cursor = conn.cursor()
start = time.time()
for object_id in range(1, 500):
  cursor.execute("select * from all_objects where object_id = " + str(object_id))
end = time.time()
print(str(end - start) + ' seconds elapsed')
# close both the cursor and the connection to free up resources
cursor.close()
conn.close()

Don't forget to take a look at the shared pool (V$SQL here). Steve will show it to you, and then clear it with:

EXEC rdsadmin.rdsadmin_util.flush_shared_pool;

Then, let's see how things are different when we use bind variables...

In [ ]:
from dotenv import load_dotenv
import os
import time

# load those values from the .env file into environment variables
load_dotenv()
# condensed example from Stage 1 revised to externalize secrets and configuration details
import oracledb
# make a connection using the environment variables (our secrets a, re no longer in the code)
conn = oracledb.connect(user=os.getenv("username"), password=os.getenv("password"), dsn=os.getenv("connect_string"))
cursor = conn.cursor()
start = time.time()
for object_id in range(1, 500):
  cursor.execute("select * from all_objects where object_id = :id", id=object_id)
end = time.time()
print(str(end - start) + ' seconds elapsed')
# close both the cursor and the connection to free up resources
cursor.close()
conn.close()

Take a look at the shared pool again and see how it's different.

# Stage 4: Connection Pooling for Scalability

In [ ]:
# no connection pool - connections will NOT be reused

from dotenv import load_dotenv
import oracledb
import threading
import os

# load those values from the .env file into environment variables
load_dotenv()

def Query():
    conn = oracledb.connect(user=os.getenv("username"), password=os.getenv("password"), dsn=os.getenv("connect_string"))
    cursor = conn.cursor()
    for i in range(4):
        cursor.execute("select sys_context('userenv','sessionid') Session_ID from dual")
        session = cursor.fetchone()
        print("Thread", threading.current_thread().name, "session = ", session[0])
    # close both the cursor and the connection to free up resources
    cursor.close()
    conn.close()

number_of_threads = 10
thread_array = []

for i in range(number_of_threads):
    thread = threading.Thread(name='#' + str(i), target=Query)
    thread_array.append(thread)
    thread.start()

for t in thread_array:
    t.join()

print("All done!")


In [ ]:
# connection pooling - connections will be reused

from dotenv import load_dotenv
import oracledb
import threading
import os

# load those values from the .env file into environment variables
load_dotenv()

pool = oracledb.create_pool(user=os.getenv("username"), password=os.getenv("password"), dsn=os.getenv("connect_string"),
                            min=2, max=10, increment=1, getmode=oracledb.POOL_GETMODE_WAIT)

def Query():
    conn = pool.acquire()
    cursor = conn.cursor()
    for i in range(4):
        cursor.execute("select sys_context('userenv','sessionid') Session_ID from dual")
        session = cursor.fetchone()
        print("Thread", threading.current_thread().name, "session = ", session[0])

number_of_threads = 10
thread_array = []

for i in range(number_of_threads):
    thread = threading.Thread(name='#' + str(i), target=Query)
    thread_array.append(thread)
    thread.start()

for t in thread_array:
    t.join()

print("All done!")


Connection pooling saves resources as it is expensive to keep opening and closing connections to the database server.